In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from rolling_ta.data import CSVLoader
from rolling_ta.volume import NumbaOBV
from rolling_ta.logging import logger
from ta.volume import OnBalanceVolumeIndicator
from time import time

INFO - Cache [numba=True]


In [3]:
xls_loader = CSVLoader()
data = xls_loader.read_resource().copy()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


In [4]:
# Precompile njit functions / Load from disk
obv = NumbaOBV(data)
obv.update(data.iloc[0])

In [5]:
%%timeit
obv = OnBalanceVolumeIndicator(data["close"], data["volume"])
obv.on_balance_volume()

260 μs ± 11.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
%%timeit
NumbaOBV(data, memory=False)
    

53.8 μs ± 1.07 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
slice_a = data.copy().iloc[:100]
slice_b = data.copy().iloc[100:]

In [8]:
%%timeit
rolling = NumbaOBV(slice_a)
for _, series in slice_b.iterrows():
    rolling.update(series)

272 ms ± 5.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
